In [2]:
import saspy
sas = saspy.SASsession(colorLOG=True)

Using SAS Config named: oda
SAS Connection established. Subprocess id is 21868



### datasets

In [76]:
# air
sdf = sas.sasdata('air', 'sashelp')
sdf.head()

,DATE,AIR
0,1949-01-01,112.0
1,1949-02-01,118.0
2,1949-03-01,132.0
3,1949-04-01,129.0
4,1949-05-01,121.0


In [77]:
sdf.columnInfo().query('Variable == "DATE"')

,Member,Num,Variable,Type,Len,Pos,Format,Label
1,SASHELP.AIR,1.0,DATE,Num,8.0,0.0,MONYY.,NaN


In [78]:
# stocks
sdf = sas.sasdata('stocks', 'sashelp')
sdf.head()

,Stock,Date,Open,High,Low,Close,Volume,AdjClose
0,IBM,2005-12-01,89.15,89.92,81.56,82.20,5976252.0,81.37
1,IBM,2005-11-01,81.85,89.94,80.64,88.90,5556471.0,88.01
2,IBM,2005-10-03,80.22,84.60,78.70,81.88,7019666.0,80.86
3,IBM,2005-09-01,80.16,82.11,76.93,80.22,5772280.0,79.22
4,IBM,2005-08-01,83.00,84.20,79.87,80.62,4801386.0,79.62


In [79]:
sdf.columnInfo().query('Variable == "Date"')

,Member,Num,Variable,Type,Len,Pos,Format,Informat,Label
2,SASHELP.STOCKS,2.0,Date,Num,8.0,0.0,DATE.,DATE.,NaN


In [80]:
# prdsale
sdf = sas.sasdata('prdsale', 'sashelp')
sdf.head()

,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
0,925.0,850.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-01-01
1,999.0,297.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-02-01
2,608.0,846.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-03-01
3,642.0,533.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,2.0,1993.0,1993-04-01
4,656.0,646.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,2.0,1993.0,1993-05-01


In [81]:
sdf.columnInfo().query('Variable.isin(["YEAR", "MONTH"])')

,Member,Num,Variable,Type,Len,Pos,Format,Label
3,SASHELP.PRDSALE,10.0,MONTH,Num,8.0,32.0,MONNAME3.,Month
9,SASHELP.PRDSALE,9.0,YEAR,Num,8.0,24.0,4.,Year


### input string as date

In [67]:
# forms that start with yy can be parsed via anydtdte10 informat.
sas.submitLST(f"""
    data df;
        input d1 anydtdte10.;
        format d1 ddmmyy10.;
        datalines;
2025-02-13
1993-02-13
1993/02/13
        ;
    run;
              
    proc print data=df;
    run;
""")

Obs,d1
1,13/02/2025
2,13/02/1993
3,13/02/1993


In [54]:
# a common form is ddmmyy10.
sas.submitLST(f"""
    data df;
        input d1 ddmmyy10.;
        format d1 ddmmyy10.;
        datalines;
13/02/2025
13-02-2025
13.02.2025
        ;
    run;
              
    proc print data=df;
    run;
""")

Obs,d1
1,13/02/2025
2,13/02/2025
3,13/02/2025


In [82]:
# the core date9 format is ddmmmyyyy
sas.submitLST(f"""
    data df;
        input d1 date9.;
        format d1 date9.;
        datalines;
13feb2025
13FEB2025
13feb40
        ;
    run;
              
    proc print data=df;
    run;
""")

sas.sasdata('df').columnInfo()

Obs,d1
1,13FEB2025
2,13FEB2025
3,13FEB1940


,Member,Num,Variable,Type,Len,Pos,Format
0,WORK.DF,1.0,d1,Num,8.0,0.0,DATE9.


### comparison to datestring

In [87]:
sas.submitLST(f"""
    proc print data=sashelp.air;
        where date <= "01mar1949"d;
    run;
""")

Obs,DATE,AIR
1,JAN49,112
2,FEB49,118
3,MAR49,132


### datefunc on dates

In [99]:
# subtracting two dates to get diff in days
# yrdif for age
# intck for month differences
sas.submitLST(f"""
    %let cutoff_date = 01mar1949;

    data res;
        set sashelp.air;
        if date <= "&cutoff_date"d;
              
        date_raw = date;
        cutoff_date = "&cutoff_date"d;
        diff_in_d = date - "&cutoff_date"d;
        yrdif = yrdif(date, "&cutoff_date"d, "AGE");
        month_boundaries = intck('month', "&cutoff_date"d, date);

        experiment = intck("month", "&cutoff_date"d, "&cutoff_date"d - 2);
        experiment2 = intck("month", "&cutoff_date"d, "&cutoff_date"d + 2);
              
        format date date9. cutoff_date date9.;
    run;
              
    proc print data=res;
    run;
""")

Obs,DATE,AIR,date_raw,cutoff_date,diff_in_d,yrdif,month_boundaries,experiment,experiment2
1,01JAN1949,112,-4017,01MAR1949,-59,0.16164,-2,-1,0
2,01FEB1949,118,-3986,01MAR1949,-28,0.07671,-1,-1,0
3,01MAR1949,132,-3958,01MAR1949,0,0.00000,0,-1,0
